In [1]:
import pandas as pd
import numpy as np
import os
import pickle
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score,confusion_matrix,classification_report
from imblearn.over_sampling import SMOTE
from sklearn.ensemble import RandomForestClassifier
import matplotlib.pyplot as plt
from sklearn.feature_selection import SelectKBest,chi2,SequentialFeatureSelector as sfs

In [2]:
pd.set_option('display.max_rows',None)
pd.set_option('display.max_columns',None)
pd.set_option('display.width',None)
pd.set_option('display.max_colwidth',None)

In [3]:
## setting work directory

In [4]:
os.chdir("U:\Propensity_Model\Model")

In [5]:
### Loading the required files

In [6]:
pickle_in = open("Final_Data_.pickle","rb")

Final_Data = pickle.load(pickle_in)

pickle_in.close()

In [7]:
### Split data into Train, Test and Validation

In [8]:
x = Final_Data.drop('AUDIT_STATUS',axis=1)
y = pd.DataFrame(Final_Data['AUDIT_STATUS'])
x_train,x_test,y_train,y_test = train_test_split(x,y,test_size=0.2,random_state=0)

In [9]:
## Checking the Selection %

In [10]:
print(y_train['AUDIT_STATUS'].value_counts()/len(y_train)*100) #1.77
print(y_test['AUDIT_STATUS'].value_counts()/len(y_test)*100)   #1.88

0    98.226685
1     1.773315
Name: AUDIT_STATUS, dtype: float64
0    98.119926
1     1.880074
Name: AUDIT_STATUS, dtype: float64


In [11]:
####Data PreProcessing (numeric columns of train_data)

##Separating out Numeric columns from train data

In [12]:
train_num = x_train.select_dtypes(include='number')

In [13]:
##PreProcessing numeric data

In [14]:
sc = StandardScaler()
train_num_process = pd.DataFrame(sc.fit_transform(train_num))
train_num_process.columns = train_num.columns
train_num_process.index = train_num.index

In [15]:
### Combining the processed data to train_data

In [16]:
col_drop = train_num.columns
x_train = x_train.drop(col_drop,axis=1)

In [17]:
## Combining the PreProcessed data to train_data

In [18]:
x_train = pd.concat([x_train,train_num_process],axis=1)

In [19]:
x_train = x_train.astype({'REV_MISSING':'Int64','REV_0636':'Int64','REV_0301':'Int64','REV_0300':'Int64','REV_0250':'Int64','REV_0305':'Int64','REV_0637':'Int64'
                            ,'REV_0260':'Int64','REV_0450':'Int64','REV_0306':'Int64','REV_0258':'Int64','REV_0420':'Int64','REV_0272':'Int64','REV_0730':'Int64',
                            'REV_0762':'Int64','REV_OTHERS':'Int64','CPT_MISSING':'Int64','CPT_83735':'Int64','CPT_80048':'Int64','CPT_96372':'Int64','CPT_96375':'Int64'
                            ,'CPT_93005':'Int64','CPT_84484':'Int64','CPT_G0378':'Int64','CPT_A9270':'Int64','CPT_85025':'Int64','CPT_80053':'Int64','CPT_36415':'Int64'
                            ,'CPT_J3490':'Int64','CPT_OTHERS':'Int64', 'REJ_MISSING':'Int64', 'REJ_0DC':'Int64', 'REJ_64V':'Int64','REJ_88':'Int64', 'REJ_0DC29T':'Int64'
                            ,'REJ_0DC46R':'Int64', 'REJ_0IREJ_04@':'Int64', 'REJ_04)':'Int64','REJ_05D':'Int64', 'REJ_26P':'Int64','REJ_97':'Int64', 'REJ_OTHERS':'Int64'})

In [20]:
## performing one-hot-encoding on categorical features

In [33]:
ctgrl_col = pd.DataFrame(x_train[['PR_NPAR','LINE_ITEM','OVR','LOB','NTWK','PRV_SPC','POT','TOB','PAR_CD','CAS_CAUSE','VCH','MBR_ST','PRIM_DX','ADMIT_DX','CLI','SUSP']])
ctgrl_cols = pd.get_dummies(ctgrl_col)

x_train = x_train.drop(ctgrl_col.columns,axis=1)
x_train = pd.concat([x_train,ctgrl_cols],axis=1)

KeyError: "None of [Index(['PR_NPAR', 'LINE_ITEM', 'OVR', 'LOB', 'NTWK', 'PRV_SPC', 'POT', 'TOB',\n       'PAR_CD', 'CAS_CAUSE', 'VCH', 'MBR_ST', 'PRIM_DX', 'ADMIT_DX', 'CLI',\n       'SUSP'],\n      dtype='object')] are in the [columns]"

In [ ]:
### Data balancing using SMOTE

In [ ]:
sm = SMOTE(sampling_strategy=0.43)

In [ ]:
x_sm,y_sm = sm.fit_resample(x_train.values,y_train.values.ravel())

In [ ]:
x_sm = pd.DataFrame(x_sm)
y_sm = pd.DataFrame(y_sm)

In [ ]:
x_sm.columns = x_train.columns
y_sm.columns = y_train.columns

In [ ]:
y_sm.value_counts(normalize=True)*100

In [ ]:
y_train.value_counts(normalize=True)*100

In [ ]:
####################################################
####################################################
####################################################

### Model Building

In [30]:
RF_v1 = RandomForestClassifier()


In [31]:
data = {
    'n_estimators':[100,200,300,400,500],
    'criterion':['gini','entropy'],
    'max_depth':[5,10,15,20,25,30],
    'min_samples_split':[1,3,5,7,9,10,12,14],
    'min_samples_leaf':[1,3,5,7,9,10,12,14],
    'max_features':['auto','sqrt','log2']
}

In [32]:
from sklearn.model_selection import RandomizedSearchCV
gscv = RandomizedSearchCV(RF_v1,data,verbose=True,n_jobs=-1)
gscv.fit(x_sm.values,y_sm.values.ravel())

Fitting 5 folds for each of 10 candidates, totalling 50 fits


C:\Users\KPA2877\.conda\envs\test1\lib\site-packages\sklearn\model_selection\_validation.py:372: FitFailedWarning: 
15 fits failed out of a total of 50.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
15 fits failed with the following error:
Traceback (most recent call last):
  File "C:\Users\KPA2877\.conda\envs\test1\lib\site-packages\sklearn\model_selection\_validation.py", line 680, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "C:\Users\KPA2877\.conda\envs\test1\lib\site-packages\sklearn\ensemble\_forest.py", line 450, in fit
    trees = Parallel(
  File "C:\Users\KPA2877\.conda\envs\test1\lib\site-packages\joblib\parallel.py", line 1043, in __call__
    if self.dispatch_one_batch(iterator):
  File "C:\Us

RandomizedSearchCV(estimator=RandomForestClassifier(), n_jobs=-1,
                   param_distributions={'criterion': ['gini', 'entropy'],
                                        'max_depth': [5, 10, 15, 20, 25, 30],
                                        'max_features': ['auto', 'sqrt',
                                                         'log2'],
                                        'min_samples_leaf': [1, 3, 5, 7, 9, 10,
                                                             12, 14],
                                        'min_samples_split': [1, 3, 5, 7, 9, 10,
                                                              12, 14],
                                        'n_estimators': [100, 200, 300, 400,
                                                         500]},
                   verbose=True)

In [34]:
gscv.best_params_

{'n_estimators': 400,
 'min_samples_split': 12,
 'min_samples_leaf': 10,
 'max_features': 'auto',
 'max_depth': 20,
 'criterion': 'gini'}

In [35]:
gscv.best_score_

0.9810636073620342

In [36]:
RF_v1 = RandomForestClassifier(n_estimators=400,min_samples_split=12,min_samples_leaf=10,max_features='auto',max_depth=20,criterion='gini')
RF_v1.fit(x_sm.values,y_sm.values.ravel())

RandomForestClassifier(max_depth=20, min_samples_leaf=10, min_samples_split=12,
                       n_estimators=400)

In [37]:
#####################################

####Data PreProcessing (numeric columns of test_data)

##Separating out Numeric columns from test data

In [38]:
test_num = x_test.select_dtypes(include='number')

In [39]:
##PreProcessing numeric data

In [40]:
test_num_process = pd.DataFrame(sc.transform(test_num))
test_num_process.columns = test_num.columns
test_num_process.index = test_num.index

In [41]:
###########
### Combining the processed data to test_data

In [42]:
col_drop = test_num.columns
x_test = x_test.drop(col_drop,axis=1)

In [43]:
## Combining the Preprocessed data to test_data

In [44]:
x_test = x_test.astype({'REV_MISSING':'Int64','REV_0636':'Int64','REV_0301':'Int64','REV_0300':'Int64','REV_0250':'Int64','REV_0305':'Int64','REV_0637':'Int64'
                            ,'REV_0260':'Int64','REV_0450':'Int64','REV_0306':'Int64','REV_0258':'Int64','REV_0420':'Int64','REV_0272':'Int64','REV_0730':'Int64',
                            'REV_0762':'Int64','REV_OTHERS':'Int64','CPT_MISSING':'Int64','CPT_83735':'Int64','CPT_80048':'Int64','CPT_96372':'Int64','CPT_96375':'Int64'
                            ,'CPT_93005':'Int64','CPT_84484':'Int64','CPT_G0378':'Int64','CPT_A9270':'Int64','CPT_85025':'Int64','CPT_80053':'Int64','CPT_36415':'Int64'
                            ,'CPT_J3490':'Int64','CPT_OTHERS':'Int64', 'REJ_MISSING':'Int64', 'REJ_0DC':'Int64', 'REJ_64V':'Int64','REJ_88':'Int64', 'REJ_0DC29T':'Int64'
                            ,'REJ_0DC46R':'Int64', 'REJ_0IREJ_04@':'Int64', 'REJ_04)':'Int64','REJ_05D':'Int64', 'REJ_26P':'Int64','REJ_97':'Int64', 'REJ_OTHERS':'Int64'})

In [45]:
##one-hot-encoding on categorical features

In [46]:
ctgrl_col = pd.DataFrame(x_test[['PR_NPAR','LINE_ITEM','OVR','LOB','NTWK','PRV_SPC','POT','TOB','PAR_CD','CAS_CAUSE','VCH','MBR_ST','PRIM_DX','ADMIT_DX','CLI','SUSP']])
ctgrl_cols = pd.get_dummies(ctgrl_col)

x_test = x_test.drop(ctgrl_col.columns,axis=1)
x_test = pd.concat([x_test,ctgrl_cols],axis=1)

In [47]:
##################
### Predicting on the test data

In [48]:
##RF_v1

In [49]:
y_pred = RF_v1.predict(x_test)
y_pred = (RF_v1.predict_proba(x_test)[:,1]>0.1).astype('int').astype('str')
print(accuracy_score(y_test,y_pred))
print(confusion_matrix(y_test,y_pred))

print(classification_report(y_test,y_pred))

C:\Users\KPA2877\.conda\envs\test1\lib\site-packages\sklearn\base.py:443: UserWarning: X has feature names, but RandomForestClassifier was fitted without feature names
  warnings.warn(


ValueError: X has 178 features, but RandomForestClassifier is expecting 192 features as input.

In [731]:
y_pred = RF_v1.predict(x_sm)
y_pred = (RF_v1.predict_proba(x_sm)[:,1]>0.5).astype('int').astype('str')
print(accuracy_score(y_sm,y_pred))
print(confusion_matrix(y_sm,y_pred))

print(classification_report(y_sm,y_pred))

C:\Users\KPA2877\.conda\envs\test1\lib\site-packages\sklearn\base.py:443: UserWarning: X has feature names, but RandomForestClassifier was fitted without feature names
  warnings.warn(
C:\Users\KPA2877\.conda\envs\test1\lib\site-packages\sklearn\base.py:443: UserWarning: X has feature names, but RandomForestClassifier was fitted without feature names
  warnings.warn(


0.9829191025180444
[[27578     7]
 [  516  2518]]
              precision    recall  f1-score   support

           0       0.98      1.00      0.99     27585
           1       1.00      0.83      0.91      3034

    accuracy                           0.98     30619
   macro avg       0.99      0.91      0.95     30619
weighted avg       0.98      0.98      0.98     30619

